# Exploring Image Processing with CONNIE Data

In [ ]:
%run ./notebook_init.py

import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from astropy.io import fits

from core import DATA_FOLDER

In [ ]:
# %matplotlib ipympl

In [ ]:
file_ext = "fits"

image_folder_path = os.path.join(DATA_FOLDER, "connie_raw_dataset")
calibrated_img_path = os.path.join(image_folder_path, "calibrated_images")
mask_path = os.path.join(image_folder_path, "mask")

In [ ]:
calibrated_img_path_list = glob(os.path.join(calibrated_img_path, "*" + file_ext))
mask_path_list = glob(os.path.join(mask_path, "*" + file_ext))

In [ ]:
calibrated_img = []
mask = []

for img in calibrated_img_path_list:
    calibrated_img.append(fits.open(img)[0])

for img in mask_path_list:
    mask.append(fits.open(img)[0])

In [ ]:
calibrated_img_list = []
for img in calibrated_img:
    calibrated_img_list.append(img.data)

In [ ]:
calibrated_img_np = np.array(calibrated_img_list)
calibrated_img_np_flatten = calibrated_img_np.flatten()
    
print(f"Shape calibrated_img_np = {np.shape(calibrated_img_np)}")
print(f"Shape calibrated_img_np_flatten = {np.shape(calibrated_img_np_flatten)}")

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(7,4))

fig.canvas.header_visible = False

axes.hist(calibrated_img_np_flatten, bins=100)
axes.set_yscale("log")
axes.set_ylabel("Number of occurrences")
axes.set_xlabel("Pixel value")
fig.suptitle("Histogram of the whole dataset")

In [ ]:
num_imgs = 10
nrows = 2
ncols = 5

In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(11.8,6.5), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.hist(calibrated_img_np[idx].flatten(), bins=100)
    ax.set_title(f"Image {idx}")
    ax.set_yscale("log")
fig.suptitle(f"Histogram of the first {num_imgs} images")
fig.tight_layout()


The masks can have 3 values, according to the table below

| Value |               Meaning               |
|-------|-------------------------------------|
| 128   | Possible Serial Register Events     | 
| 64    | Hot pixel                           |
| 0     | Pixel that will be used for analysis|

In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(10,8), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.imshow(mask[idx].data, vmin=0, vmax=128)
    ax.set_title(f"Mask {idx}")
fig.suptitle(f"Display of the first {num_imgs} masks")
fig.tight_layout()


In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(10,8), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    image_with_mask = np.ma.masked_array(calibrated_img_np[idx], mask[idx].data.astype(bool))
    print(f"Image {idx}\nMin value = {np.nanmin(image_with_mask)}".ljust(35),
          f"Max value = {np.nanmax(image_with_mask)}\n")
    ax.imshow(image_with_mask, vmin=-500, vmax=1000)
    ax.set_title(f"Image {idx}")
    fig.suptitle(f"Display of the first {num_imgs} images with mask")
fig.tight_layout()


The masks will not be used for now because some pixels in a particle track may be masked, making identification difficult.

But, we will mask the pixels corresponding to overscan (last 70 columns) + prescan (first 8 columns) 

In [ ]:
new_mask = np.full(np.shape(calibrated_img_np[0]), False)

prescan = 8
overscan = 70
frame = 5

# new_mask[:, :prescan+frame] = True
# new_mask[:, -(overscan+frame):] = True
# new_mask[:frame, :] = True
# new_mask[-frame:, :] = True

new_mask[:, :frame] = True
new_mask[:, -frame:] = True
new_mask[:frame, :] = True
new_mask[-frame:, :] = True

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(7, 5), num=1, clear=True)

data = np.ma.masked_array(calibrated_img_np[0], new_mask)

fig.canvas.header_visible = False
axes.imshow(data, vmin=-200, vmax=1000)
fig.suptitle("Masking the first image")


In [ ]:
masked_border_img = []

for img in calibrated_img_np:
    masked_border_img.append(np.ma.masked_array(img, new_mask, fill_value=np.nan))

masked_border_img_np = np.array(masked_border_img)

In [ ]:
masked_border_img[0]

## Analyzing the data

The image data is float32, with negative values included. To create the mask, we will test some processing as normalization, standalization and adding offset.

In [ ]:
def normalization(img, min_max=None, new_min=0, new_max=255):
    if min_max:
        min_value, max_value = min_max
    else:
        min_value, max_value = np.min(img), np.max(img)
    return (((img - min_value) / (max_value - min_value))*(new_max-new_min)) + new_min

In [ ]:
def standardization(img, mean_std=None):
    if mean_std:
        mean_value, std_value = mean_std
    else:
        mean_value, std_value = np.mean(img), np.std(img)
    return (img - mean_value) / std_value

In [ ]:
def offset(img):
    return img + abs(np.min(img))

As talked with Professor Irina, the negative values are not relevant in this analysis, as it do not represent a event. In the function below, we will replace the values below 0.64 with 0.

In [ ]:
def replace_negative(img):
    # 4 sigma (0.16) == 0.64
    return img.clip(0.64)

In [ ]:
test_images = masked_border_img[:num_imgs]

### Considering current image metrics

In [ ]:
calibrated_img_np_norm, calibrated_img_np_std, calibrated_img_np_offset, calibrated_img_np_clip = [], [], [], []
for img in test_images:
    calibrated_img_np_norm.append(normalization(img))
    calibrated_img_np_std.append(standardization(img))
    calibrated_img_np_offset.append(offset(img))
    calibrated_img_np_clip.append(replace_negative(img))

In [ ]:
for idx in range(num_imgs):
    print(f"Min value of calibrated_img_np_norm[{idx}] = {np.min(calibrated_img_np_norm[idx])}")
    print(f"Min value of calibrated_img_np_std[{idx}] = {np.min(calibrated_img_np_std[idx])}")
    print(f"Min value of calibrated_img_np_offset[{idx}] = {np.min(calibrated_img_np_offset[idx])}")
    print(f"Min value of calibrated_img_np_clip[{idx}] = {np.min(calibrated_img_np_clip[idx])}\n")

    print(f"Max value of calibrated_img_np_norm[{idx}] = {np.max(calibrated_img_np_norm[idx])}")
    print(f"Max value of calibrated_img_np_std[{idx}] = {np.max(calibrated_img_np_std[idx])}")
    print(f"Max value of calibrated_img_np_offset[{idx}] = {np.max(calibrated_img_np_offset[idx])}")
    print(f"Max value of calibrated_img_np_clip[{idx}] = {np.max(calibrated_img_np_clip[idx])}")
    print("---------------")

In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(10, 8), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.imshow(calibrated_img_np_norm[idx], vmin=0, vmax=50)
    ax.set_title(f"Image {idx}")
    fig.suptitle("Normalization")
fig.tight_layout()


In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(11.8,6.5), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.hist(calibrated_img_np_norm[idx].flatten(), bins=100)
    ax.set_title(f"Image {idx}")
    ax.set_yscale("log")
    fig.suptitle(f"Histogram of the first {num_imgs} images - Normalization")
fig.tight_layout()


In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(10, 8), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.imshow(calibrated_img_np_std[idx], vmin=0, vmax=5)
    ax.set_title(f"Image {idx}")
    fig.suptitle("Standalization")
fig.tight_layout()


In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(11.8,6.5), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.hist(calibrated_img_np_std[idx].flatten(), bins=100)
    ax.set_title(f"Image {idx}")
    ax.set_yscale("log")
    fig.suptitle(f"Histogram of the first {num_imgs} images - Standardization")
fig.tight_layout()


In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(10, 8), num=1, clear=True)

fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.imshow(calibrated_img_np_offset[idx], vmin=0, vmax=1000)
    ax.set_title(f"Image {idx}")
    fig.suptitle("Offset")
fig.tight_layout()


In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(11.8,6.5), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.hist(calibrated_img_np_offset[idx].flatten(), bins=100)
    ax.set_title(f"Image {idx}")
    ax.set_yscale("log")
    fig.suptitle(f"Histogram of the first {num_imgs} images - Offset")
fig.tight_layout()


Standardization it's not what we are looking for, as the negative values are kept.

In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(10, 8), num=1, clear=True)

fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.imshow(calibrated_img_np_clip[idx], vmin=0, vmax=1000)
    ax.set_title(f"Image {idx}")
    fig.suptitle("Clip negative values")
fig.tight_layout()


In [ ]:
fig, axes = plt.subplots(nrows, ncols, figsize=(11.8,6.5), num=1, clear=True)
fig.canvas.header_visible = False

for idx, ax in enumerate(axes.ravel()):
    ax.hist(calibrated_img_np_clip[idx].flatten(), bins=100)
    ax.set_title(f"Image {idx}")
    ax.set_yscale("log")
    fig.suptitle(f"Histogram of the first {num_imgs} images - Clip")
fig.tight_layout()


## Creating Masks

In [ ]:
test_img_norm = calibrated_img_np_norm[0]
test_img_clip = calibrated_img_np_clip[0]
test_img = test_img_clip

In [ ]:
binary_mask = image_contours = np.zeros((test_img.shape[0],
                                         test_img.shape[1]),
                                         np.uint8)
print(f"Shape = {np.shape(binary_mask)}")

In [ ]:
print(f"Min value = {np.min(test_img)}")
print(f"Max value = {np.max(test_img)}")
print(f"Shape = {np.shape(test_img)}")
print(test_img)

In [ ]:
num_imgs = 5

### Global thresholding

In [ ]:
factor = 3.745
threshold_ev = 10
threshold = threshold_ev/factor
print(f"Threshold in e- value equivalent to {threshold_ev}eV = {threshold}")

In [ ]:
kernel_1_1 = np.ones((1, 1), np.uint8) 
kernel_2_2 = np.ones((2, 2), np.uint8)
kernel_3_3 = np.ones((3, 3), np.uint8) 
kernel_4_4 = np.ones((4, 4), np.uint8)

kernel_2_1 = np.ones((2, 1), np.uint8) 
kernel_3_2 = np.ones((3, 2), np.uint8) 
kernel_4_2 = np.ones((4, 2), np.uint8)
kernel_5_3 = np.ones((5, 3), np.uint8) 
kernel_5_4 = np.ones((5, 4), np.uint8) 
kernel_6_1 = np.ones((6, 1), np.uint8) 
kernel_6_2 = np.ones((6, 2), np.uint8) 
kernel_6_3 = np.ones((6, 3), np.uint8) 
kernel_6_4 = np.ones((6, 4), np.uint8) 

#### Without morphological transformation

In [ ]:
fig, axes = plt.subplots(num_imgs, 3, figsize=(10, 35), num=1, clear=True)
fig.canvas.header_visible = False

for idx in range(num_imgs):
    img_clip = calibrated_img_np_clip[idx]
    img_original = calibrated_img_np_clip[idx].copy()
    axes[idx, 0].imshow(img_clip, vmin=0, vmax=1000)
    axes[idx, 0].set_title(f"Original Image {idx}")
    _, img_thresh = cv.threshold(img_clip, threshold, 255, cv.THRESH_BINARY)
    axes[idx, 1].imshow(img_thresh, vmin=0, vmax=255)
    axes[idx, 1].set_title(f"Image thresh {idx}")

    mask_inverted = ~(img_thresh).astype(np.uint8)
    contours = cv.findContours(mask_inverted, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_KCOS)[0]

    boxes = []
    for contour in contours:
        boxes.append(cv.boundingRect(contour))

    for box in boxes:
        top_left = (box[0], box[1])
        bottom_right = (box[0] + box[2], box[1] + box[3])
        cv.rectangle(img_original, top_left, bottom_right, (255,0,0), 5)
    
    axes[idx, 2].imshow(img_original, vmin=0, vmax=1000)
    axes[idx, 2].set_title(f"Image Contours {idx}")
    
fig.suptitle("Global Threshold")
fig.tight_layout()

#### With closing and opening transformation and gaussian blur

In [ ]:
fig, axes = plt.subplots(num_imgs, 3, figsize=(10, 35), num=1, clear=True)
fig.canvas.header_visible = False

for idx in range(num_imgs):
    img_clip = calibrated_img_np_clip[idx]
    img_original = calibrated_img_np_clip[idx].copy()
    axes[idx, 0].imshow(img_clip, vmin=0, vmax=1000)
    axes[idx, 0].set_title(f"Original Image {idx}")
    blurred_img = cv.GaussianBlur(img_clip, (3, 3), 0)
    _, img_thresh = cv.threshold(blurred_img, threshold, 255, cv.THRESH_BINARY)

    mask = cv.morphologyEx(img_thresh.astype(np.uint8), cv.MORPH_CLOSE, kernel_2_2)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel_2_2) 
    mask_inverted = ~mask 

    axes[idx, 1].imshow(mask_inverted, vmin=0, vmax=255)
    axes[idx, 1].set_title(f"Image mask {idx}")

    contours = cv.findContours(mask_inverted, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_KCOS)[0]

    boxes = []
    for contour in contours:
        boxes.append(cv.boundingRect(contour))

    for box in boxes:
        top_left = (box[0], box[1])
        bottom_right = (box[0] + box[2], box[1] + box[3])
        cv.rectangle(img_original, top_left, bottom_right, (255,0,0), 5)
    
    axes[idx, 2].imshow(img_original, vmin=0, vmax=1000)
    axes[idx, 2].set_title(f"Image Contours {idx}")
    
fig.suptitle("Global Threshold")
fig.tight_layout()

#### Opening (erosion + dilatation) and erosion

In [ ]:
fig, axes = plt.subplots(num_imgs, 3, figsize=(10, 35), num=1, clear=True)
fig.canvas.header_visible = False

for idx in range(num_imgs):
    img_clip = calibrated_img_np_clip[idx]
    img_original = calibrated_img_np_clip[idx].copy()
    axes[idx, 0].imshow(img_clip, vmin=0, vmax=1000)
    axes[idx, 0].set_title(f"Original Image {idx}")
    blurred_img = cv.GaussianBlur(img_clip, (3, 3), 0)
    _, img_thresh = cv.threshold(blurred_img, threshold, 255, cv.THRESH_BINARY)

    mask = cv.morphologyEx(img_thresh.astype(np.uint8), cv.MORPH_OPEN,
                           cv.getStructuringElement(cv.MORPH_CROSS,(3,3)))
    mask = cv.dilate(mask,
                     cv.getStructuringElement(cv.MORPH_CROSS,(1,1)), iterations=1)
    mask = cv.erode(mask,
                    cv.getStructuringElement(cv.MORPH_CROSS,(4, 4)), iterations=1)

    axes[idx, 1].imshow(mask, vmin=0, vmax=255)
    axes[idx, 1].set_title(f"Image mask {idx}")
    
    contours = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_KCOS)[0]

    contour_area_min = 2

    boxes = []
    for contour in contours:
        if (cv.contourArea(contour) >= contour_area_min):
            boxes.append(cv.boundingRect(contour))

    for box in boxes:
        top_left = (box[0], box[1])
        bottom_right = (box[0] + box[2], box[1] + box[3])
        cv.rectangle(img_original, top_left, bottom_right, (255,0,0), 5)
    
    axes[idx, 2].imshow(img_original, vmin=0, vmax=1000)
    axes[idx, 2].set_title(f"Image Contours {idx}")
    
fig.suptitle("Global Threshold")
fig.tight_layout()

#### Dilatation

In [ ]:
fig, axes = plt.subplots(num_imgs, 3, figsize=(10, 35), num=1, clear=True)
fig.canvas.header_visible = False

kernel = np.ones((5, 5), np.uint8) 

for idx in range(num_imgs):
    img_clip = calibrated_img_np_clip[idx]
    img_original = calibrated_img_np_clip[idx].copy()
    axes[idx, 0].imshow(img_clip, vmin=0, vmax=1000)
    axes[idx, 0].set_title(f"Original Image {idx}")
    blurred_img = cv.GaussianBlur(img_clip, (3, 3), 0)
    _, img_thresh = cv.threshold(blurred_img, threshold, 255, cv.THRESH_BINARY)

    mask = cv.dilate(img_thresh.astype(np.uint8), kernel, iterations=1)
    mask_inverted = ~mask
    axes[idx, 1].imshow(mask_inverted, vmin=0, vmax=255)
    axes[idx, 1].set_title(f"Image mask {idx}")


    contours = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_KCOS)[0]

    boxes = []
    for contour in contours:
        boxes.append(cv.boundingRect(contour))

    for box in boxes:
        top_left = (box[0], box[1])
        bottom_right = (box[0] + box[2], box[1] + box[3])
        cv.rectangle(img_original, top_left, bottom_right, (255,0,0), 5)
    
    axes[idx, 2].imshow(img_original, vmin=0, vmax=1000)
    axes[idx, 2].set_title(f"Image Contours {idx}")
    
fig.suptitle("Global Threshold")
fig.tight_layout()

#### Erosion

In [ ]:
fig, axes = plt.subplots(num_imgs, 3, figsize=(10, 35), num=1, clear=True)
fig.canvas.header_visible = False

for idx in range(num_imgs):
    img_clip = calibrated_img_np_clip[idx]
    img_original = calibrated_img_np_clip[idx].copy()
    axes[idx, 0].imshow(img_clip, vmin=0, vmax=1000)
    axes[idx, 0].set_title(f"Original Image {idx}")
    blurred_img = cv.GaussianBlur(img_clip, (3, 3), 0)
    _, img_thresh = cv.threshold(blurred_img, threshold, 255, cv.THRESH_BINARY)
    axes[idx, 1].imshow(img_thresh, vmin=0, vmax=255)
    axes[idx, 1].set_title(f"Image thresh {idx}")

    mask = cv.dilate(img_thresh.astype(np.uint8), kernel_2_1, iterations=1)
    mask = cv.erode(mask, kernel_4_2, iterations=1)
    contours = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_KCOS)[0]

    boxes = []
    for contour in contours:
        boxes.append(cv.boundingRect(contour))

    for box in boxes:
        top_left = (box[0], box[1])
        bottom_right = (box[0] + box[2], box[1] + box[3])
        cv.rectangle(img_original, top_left, bottom_right, (255,0,0), 5)
    
    axes[idx, 2].imshow(img_original, vmin=0, vmax=1000)
    axes[idx, 2].set_title(f"Image Contours {idx}")
    
fig.suptitle("Global Threshold")
fig.tight_layout()